In [4]:
from keras.datasets import imdb
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input
from keras.models import Model
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
import keras
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
! pip install glove-python-binary
from glove import Corpus, Glove

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.9/974.9 KB 56.1 MB/s eta 0:00:00


In [5]:
from pathlib import Path

import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from google_drive_downloader import GoogleDriveDownloader as gdd
from sklearn.feature_extraction.text import CountVectorizer
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm, tqdm_notebook
from sklearn.feature_extraction.text import TfidfVectorizer
# Import train_test_split function
from sklearn.model_selection import train_test_split

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
df=pd.read_csv('/content/Covid Vaccine data(3807).csv')
df.head()

,Text,Polarity
0,আমাদের সজাগ থাকতে হবে টিকা নেওয়া নিয়ে।,1
1,ভ্যাকসিন ভালো না।,0
2,আমার পরিবারের সকলেই টিকা নিয়েছে।,1
3,টিকা লক্ষণীয় রোগ প্রতিরোধে একইভাবে উচ্চ কার্য...,1
4,অনলাইনে আবেদন করে আমি ভ্যাকসিন নিয়েছি।,1


In [25]:
len(df)

3807

In [42]:
#df=pd.read_csv(DATA_PATH)
df.columns=["Text", "Polarity"]
df_train=df.head(3000)
df_test=df.tail(800)
corpus=[]
max=0
for i in range(0,len(df_train)): 
    review = df['Text'][i]
    li = list(review.split(" "))
    corpus.append(li)

In [43]:
! pip install glove-python-binary
from glove import Corpus, Glove

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [44]:
!pip install bnlp_toolkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [45]:
!pip install -U bnlp_toolkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
import gensim

/usr/local/lib/python3.8/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [46]:
model=gensim.models.Word2Vec(min_count=3,vector_size=50,workers=4,window=5)
model.build_vocab(corpus,progress_per=100)
model.train(corpus,total_examples=model.corpus_count,epochs=10)
words=list(model.wv.index_to_key)
print(words)

['টিকা', '', 'ভ্যাকসিন', 'না।', 'আমি', 'না', 'করোনা', 'জন্য', 'আমার', 'এবং', 'ডোজ', 'থেকে', 'নিতে', 'করে', 'নিয়ে', 'অনেক', 'পর', 'আর', 'আমাদের', 'টিকার', 'করা', 'এখন', 'করতে', 'নিলে', 'ও', 'ভ্যাক্সিন', 'কোন', 'এর', 'নিয়ে', 'ভেকসিন', 'মানুষ', 'এই', 'করোনার', 'হবে।', 'তৈরি', 'হবে', 'ভ্যাকসিনের', 'তাই', 'তো', 'কোনো', 'ভেক্সিন', 'মনে', 'টীকা', 'নাই।', 'ভালো', 'নেই।', 'করার', 'কি', 'যে', 'হয়ে', 'সুস্থ', 'আপনার', 'একটি', 'তাদের', 'কিছু', 'নেওয়ার', 'কাজ', 'COVID-19', 'মানুষের', 'পরে', 'পারে।', 'করে।', 'নিরাপদ', 'আগে', 'নেওয়ার', 'হয়।', 'এটা', 'বিরুদ্ধে', 'বা', 'দিতে', 'ভয়', 'বেশি', 'হয়', 'করি', 'অসুস্থ', 'আমরা', 'বুস্টার', 'হতে', 'সবাই', 'সুরক্ষা', 'নেওয়া', 'গ্রহণ', 'হচ্ছে', 'সব', 'ভাইরাস', 'প্রতিরোধ', 'যারা', 'যদি', 'দেশের', 'প্রথম', 'তা', 'দেওয়া', 'রোগ', 'দিয়ে', 'সমস্যা', 'পারে', 'নিয়েছি।', 'দেওয়া', 'দেওয়ার', 'কার্যকর', 'মধ্যে', 'বলে', 'কোভিড', 'ক্ষমতা', 'যা', 'এ', 'শরীরে', 'কে', 'বিভিন্ন', 'আছে', 'দেশে', 'কিন্তু', 'দেখা', 'হলে', 'করুন।', 'জীবন', 'রক্ষা', 'দেয়া', 'শরীরের', 'নেয়ার', 'এখ

In [47]:
model.wv.most_similar('ভ্যাকসিন')

[('স্বাস্থ্য', 0.9992122650146484),
 ('কিন্তু', 0.9991387128829956),
 ('এই', 0.9991067051887512),
 ('হচ্ছে', 0.999085545539856),
 ('কোভিড-১৯', 0.9990361928939819),
 ('কার্ড', 0.9989940524101257),
 ('সম্পর্কে', 0.9989809989929199),
 ('উচিত', 0.9989668130874634),
 ('সংক্রমণ', 0.9989615678787231),
 ('ভেক্সিন', 0.9989333152770996)]

In [48]:
new_dict=dict()
new_dict2idx=dict()
index=0
for i in words:
  x=model.wv[i]
  average = sum(x) / len(x)
  new_dict[i]=average
  new_dict2idx[i]=index #new
  index=index+1
rows, cols = (len(corpus),len(new_dict2idx)) #max len(new_dict2idx)
arr = [[0 for i in range(cols)] for j in range(rows)]
for l in range(0, len(corpus)):
    msg = corpus[l]
    #msg = msg.split()
    for j in range(0,len(msg)):
        k=msg[j]
        if k in new_dict:
          index=new_dict2idx[k] #new
          arr[l][index]=new_dict[k] #j index

In [49]:
batch_size = 100
X_train = arr
y_train = df_train['Polarity']

tensor_x = torch.Tensor(X_train) # transform to torch tensor
tensor_y = torch.Tensor(y_train)

dataset = TensorDataset(tensor_x,tensor_y) # create your datset

train_loader=DataLoader(dataset,batch_size)

In [50]:
#Hyper parameters
input_size=len(new_dict2idx) #len(dataset.token2idx) max index
hidden_size=200
num_classes=1 #Updated
num_epochs=50
sequence_length=1
num_layers=2
batch_size = 100
learning_rate=0.001

In [51]:
class RNN_Model(nn.Module):
    def __init__(self, input_size,hidden_size,num_layers,num_classes):
        super(RNN_Model,self).__init__()
        self.num_layers=num_layers
        self.hidden_size=hidden_size
        self.rnn=nn.RNN(input_size,hidden_size,num_layers,batch_first=True)
        #x -> (batch_size,seq,input_size)
        self.fc=nn.Linear(hidden_size,num_classes) 


    def forward(self, x):
        h0=torch.zeros(self.num_layers,x.size(0),self.hidden_size).to(device)
        #c0=torch.zeros(self.num_layers,x.size(0),self.hidden_size).to(device)
        
        out , _ =self.rnn(x, h0)
        #out: batch_size,seq_length,hidden_size
       
        out=out[:,-1,:]
        out=self.fc(out)
        return out

In [52]:
model=RNN_Model(input_size,hidden_size,num_layers,num_classes).to(device)

In [53]:
criterion=nn.BCEWithLogitsLoss()
optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate)
train_losses=[]
#train the model
n_total_steps=len(train_loader) 
for epoch in range(num_epochs): 
    losses=[]
    total=0
    for inputs, target in train_loader:
        #print(inputs.shape)
        inputs=inputs.reshape(batch_size,sequence_length,input_size).to(device)
        target=target.view(batch_size, -1)
        target=target.to(device) 
        model.zero_grad()
        
        output=model(inputs.float())

        loss=criterion(output, target.float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        total +=1
        #if (epoch+1)% 100 == 0:
    print(f'epoch {epoch+1}/{num_epochs}, loss={loss.item():.4f}')


epoch 1/50, loss=0.6909
epoch 2/50, loss=0.6778
epoch 3/50, loss=0.6082
epoch 4/50, loss=0.4863
epoch 5/50, loss=0.4221
epoch 6/50, loss=0.3850
epoch 7/50, loss=0.3627
epoch 8/50, loss=0.3411
epoch 9/50, loss=0.3200
epoch 10/50, loss=0.3015
epoch 11/50, loss=0.2827
epoch 12/50, loss=0.2641
epoch 13/50, loss=0.2472
epoch 14/50, loss=0.2321
epoch 15/50, loss=0.2183
epoch 16/50, loss=0.2058
epoch 17/50, loss=0.1944
epoch 18/50, loss=0.1838
epoch 19/50, loss=0.1742
epoch 20/50, loss=0.1660
epoch 21/50, loss=0.1608
epoch 22/50, loss=0.1652
epoch 23/50, loss=0.2048
epoch 24/50, loss=0.2938
epoch 25/50, loss=0.1875
epoch 26/50, loss=0.1400
epoch 27/50, loss=0.1346
epoch 28/50, loss=0.1302
epoch 29/50, loss=0.1260
epoch 30/50, loss=0.1222
epoch 31/50, loss=0.1186
epoch 32/50, loss=0.1154
epoch 33/50, loss=0.1124
epoch 34/50, loss=0.1100
epoch 35/50, loss=0.1080
epoch 36/50, loss=0.1066
epoch 37/50, loss=0.1058
epoch 38/50, loss=0.1058
epoch 39/50, loss=0.1069
epoch 40/50, loss=0.1097
epoch 41/

In [54]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [55]:
def predict_sentiment(text):
    li = list(text.split(" "))
    rows, cols = (1,len(new_dict2idx)) #new
    test_vector = [[0 for i in range(cols)] for j in range(rows)]
    for j in range(0,len(li)):
        k=li[j]
        if k in new_dict:
          index=new_dict2idx[k] #new
          test_vector[0][index]=new_dict[k] #j index
    #test_vector=torch.LongTensor(dataset.vectorizer.transform([text]).toarray())
    test_vector = torch.Tensor(test_vector)
    test_vector=test_vector.reshape(1,sequence_length,input_size).to(device)#NEW

    output=model(test_vector.float())#NEW
    prediction=torch.sigmoid(output).item()
    if prediction> 0.5:
        return 1
    else:
        return 0

pred_labels=[]
sentences=list(df_test['Text'])
labels=df_test['Polarity']
#print(sentences)
for sentence in sentences:
    pred_labels.append(predict_sentiment(sentence))
#accuracy
accuracy=accuracy_score(labels,pred_labels)
accuracy=accuracy*100
print('Accuracy: %f'% accuracy)

Accuracy: 77.625000


In [56]:
recall=recall_score(labels,pred_labels,average="binary")
recall = recall*100
print('Recall %.3f'%recall)


precision=precision_score(labels,pred_labels,average="binary")
precision = precision*100
print('precision %.3f'%precision)



f1=f1_score(labels,pred_labels,average="binary")
f1 = f1*100
print('f1_score %.3f'%f1)

Recall 85.877
precision 76.316
f1_score 80.815
